# Spark Wrangle Exercises
These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
        - There's a new folder for each json created
        - each has a number of JSON files in it. I was unable to open any of them
        - Seems like there's also a file that says SUCCESS perhaps indicating the process was completed
        
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.
    - You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder.getOrCreate()

In [2]:
schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)
# Read csv, but now we specify the schema:

source = spark.read.csv("source.csv", header=True, schema=schema)

In [3]:
# write data to a destination using .write property

source.write.json("source_json", mode="overwrite")

In [4]:
# read in case csv
case = spark.read.csv("case.csv", header=True, inferSchema=True)


In [5]:
case.write.json('case_json', mode = 'overwrite')

In [6]:
# read in dept csv

dept = spark.read.csv('dept.csv', header=True, inferSchema=True)

In [7]:
dept.write.json('dept_json', mode = 'overwrite')

In [8]:
# source seems good to go
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [9]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

Need to:
- change all dates to datetimes
    - convert any present day dates to max date in dataset
- change case late to bool
- change case closed to bool
- change council address to string

In [10]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

Need to
- get rid of dept_name, keep standardized dept name

In [11]:
# look at first three records
case.show(3, truncate=False, vertical = True)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [12]:
# Rename 'SLA_due_date' to 'case_due_date' using .withColumnRenamed

case = case.withColumnRenamed('SLA_due_date', 'case_due_date')


In [13]:
# correct data types: case_closed and case_late to boolean from Yes and No

case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|      YES|
+-----------+---------+
only showing top 5 rows



In [14]:
# use .withColumn to change columns from string to boolean values

case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [15]:
case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [16]:
# council_district as a string instead of int
case = case.withColumn('council_district', col('council_district').cast('string'))

In [17]:
# view the column

case.select('council_district').show(4)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
+----------------+
only showing top 4 rows



In [18]:
# convert case_opened_date, case_closed_date and case_due_date to datetime format

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)


+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



In [19]:
# to_timestamp, fmt

# set up a format 
fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('case_due_date', to_timestamp('case_due_date', fmt))


In [20]:
# check the three columns again

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [21]:
# lowercase and trim whitespace of address

case = case.withColumn('request_address', trim(lower(case.request_address)))

case.select('request_address').show(5, truncate = False)

+-------------------------------------+
|request_address                      |
+-------------------------------------+
|2315  el paso st, san antonio, 78207 |
|2215  goliad rd, san antonio, 78223  |
|102  palfrey st w, san antonio, 78223|
|114  la garde st, san antonio, 78223 |
|734  clearview dr, san antonio, 78228|
+-------------------------------------+
only showing top 5 rows



In [22]:
# create a new column for zipcode:

case = case.withColumn('zipcode', regexp_extract('request_address', r"(\d+$)", 1))

case.select('zipcode').show(5)

+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
|  78223|
|  78228|
+-------+
only showing top 5 rows



In [23]:
case.select('case_due_date').agg(max('case_due_date')).show(10)

+-------------------+
| max(case_due_date)|
+-------------------+
|2022-05-27 15:07:00|
+-------------------+



In [24]:
# looking at dates to see if there's any past present day

case.select('case_due_date', 'case_opened_date' ,'case_status', 'service_request_type').sort(desc('case_due_date')).show(10, truncate = False)


+-------------------+-------------------+-----------+--------------------------------------+
|case_due_date      |case_opened_date   |case_status|service_request_type                  |
+-------------------+-------------------+-----------+--------------------------------------+
|2022-05-27 15:07:00|2018-07-27 15:07:00|Open       |Signal Timing Modification By Engineer|
|2022-05-25 15:42:00|2018-07-25 15:42:00|Open       |Signal Timing Modification By Engineer|
|2022-05-24 10:01:00|2018-07-24 10:01:00|Open       |Engineering Design                    |
|2022-04-20 10:43:00|2018-06-19 10:43:00|Open       |Signal Timing Modification By Engineer|
|2022-02-24 08:30:00|2018-04-22 20:27:00|Open       |Signal Timing Modification By Engineer|
|2022-02-15 11:07:00|2018-04-12 11:07:00|Open       |Signal Timing Modification By Engineer|
|2022-02-03 08:30:00|2018-04-01 12:28:00|Open       |Signal Timing Modification By Engineer|
|2021-12-22 16:59:00|2018-02-16 16:59:00|Open       |Signal Timing Mod

In [25]:
# create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [26]:
case.show(3, truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 2018-01-01 00:42:00                   
 case_closed_date     | 2018-01-01 12:29:00                   
 case_due_date        | 2020-09-26 00:42:00                   
 case_late            | false                                 
 num_days_late        | -998.5087616000001                    
 case_closed          | true                                  
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  el paso st, san antonio, 78207  
 council_district     | 5                                     
 zipcode              | 78207                          

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
-  The oldest case in terms of days past SLA is 1419 days old

- Cases that have been open the longest have been open for 1678 days

SLA days is due_date - start_date

In [107]:
case.filter(col('case_status') == 'Open')\
.select('SLA_days', 'case_status', 'case_age', 'case_due_date', 'case_opened_date', 'case_closed_date').sort(desc('SLA_days')).show(10)

+-----------+-----------+--------+-------------------+-------------------+----------------+
|   SLA_days|case_status|case_age|      case_due_date|   case_opened_date|case_closed_date|
+-----------+-----------+--------+-------------------+-------------------+----------------+
| 1419.00191|       Open|    1414|2021-08-11 08:30:00|2017-09-22 08:27:00|            null|
|     1419.0|       Open|    1372|2021-09-22 14:05:00|2017-11-03 14:05:00|            null|
|1416.063981|       Open|    1354|2021-10-07 08:30:00|2017-11-21 06:57:00|            null|
|1414.813009|       Open|    1342|2021-10-18 08:30:00|2017-12-03 12:59:00|            null|
|1406.003623|       Open|    1673|2020-11-12 08:30:00|2017-01-06 08:24:00|            null|
|     1406.0|       Open|    1673|2020-11-12 10:23:00|2017-01-06 10:23:00|            null|
|     1406.0|       Open|    1673|2020-11-12 08:41:00|2017-01-06 08:41:00|            null|
|     1406.0|       Open|    1673|2020-11-12 10:21:00|2017-01-06 10:21:00|      

In [28]:
case.filter(col('case_status') == 'Open')\
.select('SLA_days', 'case_status', 'case_age', 'case_due_date', 'case_opened_date', 'case_closed_date').sort(desc('case_age')).show(10)

+-----------+-----------+--------+-------------------+-------------------+----------------+
|   SLA_days|case_status|case_age|      case_due_date|   case_opened_date|case_closed_date|
+-----------+-----------+--------+-------------------+-------------------+----------------+
|15.77859954|       Open|    1678|2017-01-17 08:30:00|2017-01-01 13:48:00|            null|
|15.77247685|       Open|    1678|2017-01-17 08:30:00|2017-01-01 13:57:00|            null|
|       15.0|       Open|    1677|2017-01-17 11:26:00|2017-01-02 11:26:00|            null|
|       15.0|       Open|    1676|2017-01-18 10:01:00|2017-01-03 10:01:00|            null|
|      128.0|       Open|    1675|2017-05-12 14:55:00|2017-01-04 14:55:00|            null|
|      128.0|       Open|    1675|2017-05-12 12:19:00|2017-01-04 12:19:00|            null|
|      128.0|       Open|    1675|2017-05-12 15:02:00|2017-01-04 15:02:00|            null|
|      128.0|       Open|    1675|2017-05-12 10:56:00|2017-01-04 10:56:00|      

### 2. How many Stray Animal cases are there?
- 26760

In [29]:
case[case.service_request_type == 'Stray Animal'].count()

26760

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

- 116915 are not classified as Officer Standby


In [30]:
case[case.dept_division == 'Field Operations'].groupby('service_request_type').count().show(truncate = False)

+-------------------------------+-----+
|service_request_type           |count|
+-------------------------------+-----+
|Animal Permits Request         |3026 |
|Injured Animal(Critical)       |9633 |
|Officer Standby                |3013 |
|Animal Bite(Non-Critical)      |4750 |
|Stray Animal                   |26760|
|Trapped/Confined Animal        |11354|
|City Council Animal Request    |365  |
|Aggressive Animal(Non-Critical)|24882|
|Animal Bite(Critical)          |708  |
|Public Nuisance(Own Animal)    |10715|
|Aggressive Animal(Critical)    |5266 |
|Animal Cruelty(Critical)       |3001 |
|Spay/Neuter Request Response   |1    |
|Animal Neglect                 |13441|
+-------------------------------+-----+



Looks like a bunch of them arent classified as Officer Standby

In [31]:
case[case.dept_division == 'Field Operations'].select(col('service_request_type') != 'Officer Standby').count()

116915

### 4. Convert the council_district column to a string column.



In [32]:
# use format_string function to pad zeros for council_district

case = case.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [33]:
case.select('council_district').show(5)

+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
|             003|
|             007|
+----------------+
only showing top 5 rows



5. Extract the year from the case_closed_date column.



In [34]:
case.select(year('case_closed_date')).show(10)

+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 10 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.



In [35]:
case.withColumn('num_hours_late', expr('num_days_late * 24')).select('num_days_late', 'num_hours_late').show(10)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
+-------------------+-------------------+
only showing top 10 rows



In [39]:
# reassign case variable with num_hours_kate
case = case.withColumn('num_hours_late', expr('num_days_late * 24')).select('*')

In [40]:
case.show(3, truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 2018-01-01 00:42:00                   
 case_closed_date     | 2018-01-01 12:29:00                   
 case_due_date        | 2020-09-26 00:42:00                   
 case_late            | false                                 
 num_days_late        | -998.5087616000001                    
 case_closed          | true                                  
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  el paso st, san antonio, 78207  
 council_district     | 005                                   
 zipcode              | 78207                          

### 7. Join the case data with the source and department data.

In [86]:
# only need to keep 'standardized dept_name' when we join
dept.show(1)

+---------------+----------------+----------------------+-------------------+
|  dept_division|       dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+----------------+----------------------+-------------------+
|311 Call Center|Customer Service|      Customer Service|                YES|
+---------------+----------------+----------------------+-------------------+
only showing top 1 row



In [54]:
# join the df and dept dataframe using 'dept_division' for the key
# keep only standardized_dept_name
# convert dept_subject_to_SLA to boolean

df = (
    case
    # do a left join keeping everything in the case dataframe
    .join(dept, "dept_division", "left")
    # drop deptartment columns we don't need
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [110]:
source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [111]:
# join source table
df = df.join(source, on = df.source_id== source.source_id).drop(source.source_id)

In [112]:
df.show(2, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 zipcode              | 78207                                
 case_age             | 1313                                 
 days_to

### 8. Are there any cases that do not have a request source?
- 0
- tried different methods of sorting to see if there were any empty values, null values, or ones with a string that said null

In [56]:
df.groupby('source_id').count().sort(desc('source_id')).show(50)

+---------+------+
|source_id| count|
+---------+------+
|  yh24110|    21|
|  yc16753| 15080|
|  vk26526|     6|
|  vb22265|  2021|
|  ts15690|   146|
|  sw26367|    95|
| svcCRMSS|113585|
| svcCRMLS| 18445|
| svcCFlag|   673|
|  sv24848|   492|
|  ss26317|     5|
|  ss21394| 15607|
|  ss09159| 13076|
|  sp26368|   124|
|  sg26196|     2|
|  sg22264|  1841|
|  ru26699|    45|
|  rs16746| 11111|
|  rb05270|   437|
|  ps01944|   213|
|  ns16326| 17234|
|  np26458|    18|
|  mt26294|    64|
|  mt23152|  7452|
|  mt13131| 19570|
|  mp26640|    93|
|  mp21218| 15817|
|  mg26820|    19|
|  me05816| 19840|
|  mc21309| 15747|
|  ls26247|    17|
|  lg26383|     9|
|  ld04403|   226|
|  lb24838|     1|
|  lb07325|     3|
|  jw10936| 15088|
|  js26451|     3|
|  js12254|    99|
|  jg06389|    42|
|  ih24384|    33|
|  gp20685| 17556|
|  gg14829|    78|
|  gg02947|     2|
|  em25724|    70|
|  eg16317|   251|
|  ec25702|     1|
|  dv26072|    45|
|  dl09223|    18|
|  dl05036| 13481|
|  df03076| 

### 9. What are the top 10 service request types in terms of number of requests?
   - No Pickup                        
   - Overgrown Yard/Trash            
   - Bandit Signs                    
   - Damaged Cart                    
   - Front Or Side Yard Parking      
   - Stray Animal                    
   - Aggressive Animal(Non-Critical) 
   - Cart Exchange Request           
   - Junk Vehicle On Private Property
   - Pot Hole Repair                 

In [58]:
df.groupBy('service_request_type').count().sort(desc('count')).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?
- Zoning: Recycle Yard                  
- Zoning: Junk Yards                    
- Structure/Housing Maintenance         
- Donation Container Enforcement        
- Storage of Used Mattress              
- Labeling for Used Mattress            
- Record Keeping of Used Mattresses     
- Signage Requied for Sale of Used Mattr
- Traffic Signal Graffiti               
- License Requied Used Mattress Sales   


In [71]:
# group by service type request, 
df.filter(col('case_late') == 'true')\
.groupBy('service_request_type').agg(mean('num_days_late').alias('avg_days_late')).sort(desc('avg_days_late')).show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg_days_late     |
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



### 11. Does number of days late depend on department?
- yes customer service has a much higher average days late than the other departments.
- This could be because it is a catchall department and most things go there.

In [81]:
df.filter(col('case_late') == 'true')\
.groupby('department').agg(round(mean('num_days_late'),2).alias('avg_days_late'))\
.sort(desc('avg_days_late')).show(truncate = False)

+------------------------+-------------+
|department              |avg_days_late|
+------------------------+-------------+
|Customer Service        |87.68        |
|DSD/Code Enforcement    |49.38        |
|Animal Care Services    |23.46        |
|Parks & Recreation      |22.35        |
|Trans & Cap Improvements|10.6         |
|Solid Waste             |7.19         |
|Metro Health            |6.54         |
+------------------------+-------------+



### 12. How do number of days late depend on department and request type?
- Even though you might expect customer service to have the highest avg days late when sorted by department and service request type, it doesn't. It is behind many other service request types from other departments. When customer service type does show up it's with teh service request type 'complaint.' The average is pretty close the same as well, 87.74. I believe this might be a catch all for general complaints. There are only two different request types in this category. Complaints and Compliments
- The variance in the different types of service request types for the DSD/Code Enforcement seems like that is what's bringing the overall average late days down. There are over 100 different service request types in this department

In [90]:
(
    # Filter by if the case is late
    df.filter(col('case_late') == 'true') 

    # group by department and service request type
    .groupby('department', 'service_request_type')
    
    # get average of the number of days late column, and round 
    .agg(round(mean('num_days_late'),2).alias('avg_days_late'))
    
    # sort by descending average days late
    .sort(desc('avg_days_late')).show(25, truncate = False)
)

+------------------------+----------------------------------------+-------------+
|department              |service_request_type                    |avg_days_late|
+------------------------+----------------------------------------+-------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |210.89       |
|DSD/Code Enforcement    |Zoning: Junk Yards                      |200.21       |
|DSD/Code Enforcement    |Structure/Housing Maintenance           |190.21       |
|DSD/Code Enforcement    |Donation Container Enforcement          |171.09       |
|DSD/Code Enforcement    |Storage of Used Mattress                |163.97       |
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.43       |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |154.0        |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.64       |
|Trans & Cap Improvements|Traffic Signal Graffiti                 |137.65       |
|DSD/Code Enforc

In [100]:
(df[df.department == 'Customer Service'].groupby('service_request_type').count().sort(desc('count')).show(truncate = False)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Complaint           |2388 |
|Compliment          |461  |
+--------------------+-----+



Notice how many different types of service requests there are. 

Notice the range of average days late on the service request types

In [105]:
(
    # get all entries in code enforcement
    df[df.department == 'DSD/Code Enforcement']
    
    # filter for if the case is late
    .filter(col('case_late') == 'true')
    .groupby('service_request_type')
    
    # get the avg days late and count of service types
    .agg(round(mean('num_days_late'),2).alias('avg_days_late'), count('service_request_type').alias('count'))
    
    # sort by avg days late 
    .sort(desc('avg_days_late'))
    .show(125, truncate = False)

)

+--------------------------------------------+-------------+-----+
|service_request_type                        |avg_days_late|count|
+--------------------------------------------+-------------+-----+
|Zoning: Recycle Yard                        |210.89       |132  |
|Zoning: Junk Yards                          |200.21       |262  |
|Structure/Housing Maintenance               |190.21       |51   |
|Donation Container Enforcement              |171.09       |122  |
|Storage of Used Mattress                    |163.97       |7    |
|Labeling for Used Mattress                  |162.43       |7    |
|Record Keeping of Used Mattresses           |154.0        |7    |
|Signage Requied for Sale of Used Mattr      |151.64       |12   |
|License Requied Used Mattress Sales         |128.8        |7    |
|Curbstoning                                 |120.4        |123  |
|Vendors                                     |117.73       |131  |
|CPS Energy Wood Poles                       |115.19       |1 